In [ ]:
!pip install cuda-python

#Preprocessing Data

In [ ]:
import numpy as np
import pandas as pd


# calculate the number of events associated with each item
# returns a dictionary {aid: count_events}
def cal_eventsperitem(filepath):
    chunk_size = 10000

    aid_counts = {}

    chunks = pd.read_json(filepath, lines=True, chunksize=chunk_size)
    for i, chunk in enumerate(chunks):
        events = chunk['events']
        events_exploded = events.explode('events')
        aid_values = events_exploded.apply(lambda x: x['aid'])

        aid_count = aid_values.value_counts()
        for key, value in aid_count.items():
            if key in aid_counts:
                aid_counts[key] += value
            else:
                aid_counts[key] = value
    return aid_counts


file_path = "/kaggle/input/otto-recommender-system/train.jsonl"
aidcounts = cal_eventsperitem(file_path)
print(len(aidcounts))

# get a list of aids that should be filtered out
# save this list to to_remove.csv file
total_n_events = sum(aidcounts.values())
freqency_limit = 10

words = [word for word, freq in aidcounts.items() if freq < freqency_limit]
freqs = [freq for word, freq in aidcounts.items() if freq < freqency_limit]

aid_fre_low = {"word": words, "freq": freqs}
df = pd.DataFrame(aid_fre_low)
df.to_csv("to_remove.csv")


In [ ]:
#remove all the items with low frequency
#ori_filepath: the path to the origianl train data file
#filtered_filepath: the path to the file to save the filtered train data
#aid_to_remove: a list of items to be removed from the original data

def filterdata(ori_filepath, filtered_filepath, aid_to_remove):
    chunk_size = 10000
    sessioncount = 0
    chunks = pd.read_json(ori_filepath, lines=True, chunksize=chunk_size)
    for i, chunk in enumerate(chunks):
        # only remove events, not sessions
        # exp_chunk = chunk.explode('events')
        # exp_filter_chunk = exp_chunk[~exp_chunk['events'].apply(lambda x: x['aid'] in aid_to_remove.keys())]
        # result = exp_filter_chunk.groupby('session').agg({'events':lambda x: list(x)})
        # result = result.reset_index(drop=False)
        #remove all sessions with events in to_be_removed
        exp_filter_chunk = chunk[~chunk['events'].apply(lambda x: any(d['aid'] in aid_to_remove.keys() for d in x))]
        result = exp_filter_chunk
        #covert the data frame to JSONL format

        sessioncount += result.shape[0]
        jsonl_str = result.to_json(orient='records',lines=True)
        with open(filtered_filepath, 'a') as f:
            f.write(jsonl_str + '\n')
    return sessioncount
ori_filepath = "/kaggle/input/otto-recommender-system/train.jsonl"
filtered_filepath = "/kaggle/temp/filtered_train.jsonl"

csv_pd = pd.read_csv("/kaggle/working/to_remove.csv")
aid_to_remove = dict(zip(csv_pd['word'], csv_pd['freq']))
sess_count = filterdata(ori_filepath, filtered_filepath,aid_to_remove)
print(sess_count)


In [ ]:
import os
# get the encoding for the aid, the new value of aid from should be from 0 to # of aids..
# return two dictionaries {oldaid: newaid} {newaid: oldaid}
def encode_aid(filepath):
    aidnumencoding = {}
    numaidencoding = {}
    newid = 1
    chunk_size = 10000
    chunks = pd.read_json(filepath, lines=True, chunksize=chunk_size)
    for i, chunk in enumerate(chunks):
        aids = chunk['events'].apply(lambda x: [d['aid'] for d in x])
        for row in aids:
            for aid in row:
                if aid not in aidnumencoding.keys():
                    numaidencoding[newid] = aid
                    aidnumencoding[aid] = newid
                    newid += 1
    return aidnumencoding, numaidencoding


aidnumencoding, numaidencoding = encode_aid("/kaggle/input/cleaneddata/filtered_train.jsonl")
# os.remove("/kaggle/working/filtered_train.jsonl")

aids = aidnumencoding.keys()
nums = aidnumencoding.values()
aidnum = {'aids': aids, 'nums': nums}
df = pd.DataFrame(aidnum)
df.to_csv('aid_num_encoding.csv')

nums = numaidencoding.keys()
aids = numaidencoding.values()
numaid = {'nums': nums, 'aids': aids}
df = pd.DataFrame(numaid)
df.to_csv('num_aid_encoding.csv')


#get the encoding dictionary from the csv file
codingpath = '/kaggle/input/aid-num-encoding/aid_num_encoding.csv'
df = pd.read_csv(codingpath)
df = df[['aids','nums']]
aidnumencoding = dict(zip(df['aids'],df['nums']))
print(len(aidnumencoding))


In [ ]:
#split the train dataset into different files according to the length of the event sequence
#encode the item id into 1, 2, 3, ...
def encoding(filepath, aidnumencoding):
    type_encoding = {'clicks': 1, 'carts': 2, 'orders': 3}
    length_ranges = list(range(5, 45, 5))
    length_ranges_long = list(range(50, 550, 50))
    length_ranges += length_ranges_long

    file_seq_count = {}
    for i, max_length in enumerate(length_ranges):
        if i == 0:
            min_length = 0
        else:
            min_length = length_ranges[i - 1]
        seq_count = 0
        event_seqs = []
        chunks = pd.read_json(filepath, lines=True, chunksize=10000)
        for chunk in chunks:
            for row in chunk.to_numpy():
                session_id, events = row
                if min_length < len(events) <= max_length:
                    event_items = [aidnumencoding[event['aid']] for event in events]
                    event_actions = [type_encoding[event['type']] for event in events]
                    n_paddings = max_length - len(events)
                    event_items = [0] * n_paddings + event_items
                    event_actions = [0] * n_paddings + event_actions
                    event_seqs.append(np.array([event_items, event_actions]))
                    seq_count += 1
        #             break
        event_seqs_np = np.array(event_seqs)

        np.save(f"./preprocessed/seq_{max_length}", event_seqs_np)

        file_seq_count[max_length] = seq_count

    print(file_seq_count)


filepath = "/kaggle/input/cleaneddata/filtered_train.jsonl"
encoding(filepath, aidnumencoding)


In [ ]:
#randomly split each train file into train data and validation data by 8:2
def split_train_validate(file_paths):
    for file in file_paths:
        sessions = np.load(file)
        size = sessions.shape[0]
        print(f"size of the file {size}")
        # Randomly select N items
        selected_indices = np.random.choice(sessions.shape[0], size= int(size * 0.2), replace=False)
        selected_sessions = sessions[selected_indices]
        print(f"size of selected sessions {selected_sessions.shape[0]}")
        # Remove selected items from original array
        remaining_sessions = np.delete(sessions, selected_indices, axis=0)
        print(f"size of remaining sessions {remaining_sessions.shape[0]}")
        filename = os.path.splitext(os.path.basename(file))[0]
        np.save(f"./preprocessed/train_{filename}", remaining_sessions)
        np.save(f"./preprocessed/validation_{filename}", selected_sessions)

# Example directory path
dir_path = "/kaggle/input/cleaned-data/"

# Get file paths of all files in directory tree
file_paths = []
for root, dirs, files in os.walk(dir_path):
    for file in files:
        file_paths.append(os.path.join(root, file))
# os.makedirs("./preprocessed")
split_train_validate(file_paths)

#Metrics

In [ ]:
import torch
from torch import Tensor
import numpy as np


def ndcg_at_ip(predictions: Tensor, positive_index: Tensor, k: Tensor):
    """
    Computes the discounted cumulative gain (DCG) at a given value of k.

    Args:
    - scores: a tensor of relevance scores (in descending order) of recommended items.
    - positive_index: a positive integer specifying the number of top items to be considered.

    Returns:
    - ndcg: a value of the DCG score.
    """
    dcg = 0.0
    for i, p in enumerate(predictions[:k]):
        if p == positive_index:
            dcg += 1.0 / float(np.log2(i + 2.0))

    return dcg

def ndcg_batch(scores: Tensor, positive_batch: Tensor):
    batch_dcg = 0
    for i in range(len(positive_batch)):
        top_n = torch.topk(scores[i, :], 100).indices.tolist()
        score = ndcg_at_ip(top_n, positive_batch[i], 20)
        batch_dcg += score
    return batch_dcg

#Data Loader

In [ ]:
from torch.utils.data import IterableDataset
from torch.utils.data import DataLoader
import random
import pandas as pd
import numpy as np
import os

#parameter
negative_sample_size = 1
num_item = 364846 + 2
class JsonlDataset(IterableDataset):
    def __init__(self, filepaths):
        self.filepaths = filepaths
        random.shuffle(self.filepaths)
        # self.files = [pd.read_json(filepath, lines=True) for filepath in self.filepaths]

    def __iter__(self):
        return self

    def __next__(self):
        for file in self.filepaths:
            json_df = pd.read_json(file, lines=True)
            json_df['events'] = json_df['events'].apply(lambda x: [[d['ts'], d['type'], d['aid']] for d in x])
            print(json_df.head())
            numdata = json_df.to_numpy()
            random.shuffle(numdata)
            yield numdata

class NumpyDataset(IterableDataset):
    def __init__(self, filepaths, batch_size):
        self.filepaths = filepaths
        self.batch_size = batch_size
        random.shuffle(self.filepaths)

    def __iter__(self):
        return self

    def __next__(self):
        for file in self.filepaths:
            data = np.load(file)
            random.shuffle(data)
            batches = np.array_split(data, len(data) // self.batch_size)
            for batch in batches:
                yield {"items": batch, "negatives": np.random.randint(1, num_item,
                                                                      size=(len(batch),
                                                                            len(batch[0][0]),
                                                                            negative_sample_size))}



#Transformer Model

In [ ]:
import pandas as pd
import numpy as np
from torch import nn, Tensor
import torch


#the function to pad the input sequence
from torch.utils.data import IterableDataset

def generate_encoder_input_mask(dim: int, batchsize: int, num_heads: int) -> Tensor:
    """
    Args:
        dim: int, for src masking this must be encoder sequence length (i.e.
              the length of the input sequence to the model)
    Return:
        A Tensor of shape (batch_size, num_heads, dim, dim)
    """
    mask = torch.triu(torch.ones(dim, dim) * float('-inf'), diagonal=1)
    # mask = mask.repeat(batchsize * num_heads, 1, 1)
    return mask



In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import io
import math
import torch
from typing import Optional, Any, Union, Callable
import torch.nn as nn
from torch.nn.modules.normalization import LayerNorm

from torch import Tensor
import torch.nn.functional as F



# define the transformer model
class TimeSeriesTransformer(nn.Module):
    """
        args:

        shapes:
    """
    def __init__(self,
                 num_item,
                 output_size,
                 hidden_size,
                 num_layers=4,
                 num_heads=4,
                 dim_feedforward_encoder=256,
                 dropout=0.2,
                 max_length=500,
                 layer_norm_eps: float = 1e-5,
                 device = None,
                 dtype = None
                 ):
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.nhead = num_heads

        self.embedding_aid = nn.Embedding(num_embeddings=num_item + 1, embedding_dim=hidden_size, padding_idx=0, **factory_kwargs)
        self.embedding_type = nn.Embedding(num_embeddings=4, embedding_dim=hidden_size, padding_idx=0, **factory_kwargs)
        self.pos_embedding = nn.Embedding(num_embeddings=max_length, embedding_dim=hidden_size, **factory_kwargs)
        self.device = device
        encoderlayer = nn.TransformerEncoderLayer(
            d_model=hidden_size,
            nhead=num_heads,
            dim_feedforward=dim_feedforward_encoder,
            dropout=dropout,
            batch_first=True,
            **factory_kwargs
            )
        encoder_norm = LayerNorm(hidden_size, **factory_kwargs)
        self.encoder = nn.TransformerEncoder(encoder_layer=encoderlayer, num_layers=num_layers, norm=encoder_norm)

        self.fc_type = nn.Linear(hidden_size, 3, **factory_kwargs)


    def forward(self, src_aid: Tensor, src_type: Tensor,
                src_mask: Tensor,
                src_key_padding_mask: Optional[Tensor] = None):
        src_padding_mask = (src_aid == 0)
        src_padding_mask = src_padding_mask.to(src_mask.dtype)
        src_padding_mask = src_padding_mask.to(device)
#         print(f"src_type: {src_type}")
        src_aid_embedding = self.embedding_aid(src_aid)
        src_type_embedding = self.embedding_type(src_type)
        src = src_aid_embedding * src_type_embedding

        position_ids = torch.arange(0, src_aid.shape[-1])
        position_ids = position_ids.unsqueeze(0).to(device)
#         print(position_ids.device)
        position = self.pos_embedding(position_ids)

        src = src + position

        aid_type_embedding = self.encoder(src, mask=src_mask, src_key_padding_mask=src_padding_mask)

        output_type = self.fc_type(aid_type_embedding)
        # Apply the softmax activation function
        output_type = F.softmax(output_type, dim=1)

        return aid_type_embedding, output_type


    def predict(self, src_aid: Tensor, src_type: Tensor,
                src_mask: Optional[Tensor] = None,
                src_key_padding_mask: Optional[Tensor] = None):
        aid_type_embedding, type_probs = self(src_aid, src_type, src_mask=src_mask)

        last_event_embeddings = aid_type_embedding[:, aid_type_embedding.size(1) - 1:, :]
        last_event_embeddings = last_event_embeddings.reshape(-1, last_event_embeddings.size(2))
        aid_scores = torch.matmul(last_event_embeddings, self.embedding_aid.weight.permute(1, 0)).squeeze(1)

        type_probs = type_probs[:, type_probs.size(1) - 1:, :]
        type_probs = type_probs.reshape(-1, type_probs.size(2))

        return aid_scores, type_probs

    def train_step(self, src_aid: Tensor, src_type: Tensor,
                    negative_samples: Tensor,
                    src_mask: Optional[Tensor] = None):

        aid_type_embedding, output_type = self(src_aid, src_type, src_mask)

        target_type = src_type[:, 1:].reshape(-1)
        target_type = torch.nn.functional.one_hot(target_type, num_classes=4)
        target_type = target_type[:, 1:]# remove the 0 class

        output_type = output_type[:, :-1, :]#during training, remove the predicted last token
        output_type = output_type.reshape(-1, output_type.size(2))
        aid_type_embedding = aid_type_embedding[:, :-1, :]#during training, remove the attended last token
        aid_type_embedding = aid_type_embedding.reshape(-1, aid_type_embedding.size(2))  # (batch size * sequence_length, embedding dimension)

        positives = src_aid[:, 1:]
        positives_embeddings = self.embedding_aid(positives)# (batch size, sequence_length, hidden_size)
        positives_embeddings = positives_embeddings.reshape(-1, positives_embeddings.size(2))  # (batch size * sequence_length)
        negative_samples = negative_samples[:, :-1]  # (batch size, sequence_length)
        negatives = negative_samples.reshape(-1)  # (batch size * sequence_length)
        negatives_embeddings = self.embedding_aid(negatives)# (batch size * sequence_length, embedding dimension)
        positives_scores = (positives_embeddings * aid_type_embedding).sum(1)  # (batch size * sequence_length)
        negatives_scores = (negatives_embeddings * aid_type_embedding).sum(1)  # (batch size * sequence_length)

        is_target = (positives != 0).float().reshape(-1)  # (batch size * sequence_length)
        aid_loss = (- torch.log(torch.sigmoid(positives_scores) + 1e-24) * is_target
                    - torch.log(1 - torch.sigmoid(negatives_scores) + 1e-24) * is_target)
        type_loss = - target_type * torch.log(output_type)
        loss = (aid_loss.sum() + type_loss.sum()) / (is_target.sum() * 2)

        return {'loss': loss}

    def validation_step(self, src_aid: Tensor, src_type: Tensor,
                   src_mask: Optional[Tensor] = None):
        aid_scores, type_probs = self.predict(src_aid, src_type, src_mask = src_mask)
        aid_positive = src_aid[:, src_aid.size(1) - 1:].reshape(-1) #(batch size )

        dcg = ndcg_batch(aid_scores, aid_positive)

        # Get the maximum number in the second dimension (axis=1)
        type_prediction = torch.argmax(type_probs, dim=1)
        type_positive = src_type[:, src_type.size(1) - 1:].reshape(-1) #(batch size )
        correct = (type_prediction == type_positive).sum()
        # accuracy = ( correct / type_positive.size(0)) * 100

        return dcg, correct

    def test_step(self, src_aid: Tensor, src_type: Tensor,
                        negative_samples: Tensor,
                        src_mask: Optional[Tensor] = None):
        aid_scores, type_probs = self.predict(src_aid, src_type, src_mask)

        return None


#Train and Evaluation

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import time
import os
import gc

# Define the hyperparameters
input_size = 364846 + 1
output_size = 364846 + 1
num_heads = 4
hidden_size = 32 * num_heads
num_layers = 2
dropout = 0.2
max_seq_length = 500 + 1
batch_size = 128
lr = 0.001
num_epochs = 5

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Create the model and the optimizer
model = TimeSeriesTransformer(num_item=input_size,
                              output_size=output_size,
                              hidden_size=hidden_size,
                              num_heads=num_heads,
                              num_layers=num_layers,
                              dropout=dropout,
                              max_length=max_seq_length,
                              device=device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#loading a checkpoint
checkpoint = torch.load('checkpoint_0_.pt')
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

criterion = nn.CrossEntropyLoss()
start_time = time.time()
total_loss = 0


# for root, dirs, files in os.walk('/kaggle/input/train-data/'):
#     for file in files:
#         trian_filepaths.append(os.path.join(root, file))
train_filepaths = []
train_filepaths = ['train_seq_5.npy','train_seq_10.npy', 'train_seq_15.npy', 'train_seq_20.npy', 'train_seq_25.npy',
                  'train_seq_30.npy', 'train_seq_35.npy', 'train_seq_40.npy', 'train_seq_50.npy', 'train_seq_100.npy', 'train_seq_150.npy']
train_dataset = NumpyDataset(train_filepaths, batch_size)

for epoch in range(num_epochs):
    train_dataset = NumpyDataset(train_filepaths, batch_size)
    batches = next(iter(train_dataset))
    print(f"number of train batches:")
    num_batch = 0
    for batch in batches:
        event_sequences = batch['items']
        negative_samples = torch.from_numpy(batch['negatives']).to(device)
        if len(event_sequences) == 1:
          print("break")
          break
        aids_input = torch.from_numpy(np.array([event_sequence[0] for event_sequence in event_sequences]))
        types_input = torch.from_numpy(np.array([event_sequence[1] for event_sequence in event_sequences]))
        aids_input = aids_input.to(device)
        types_input = types_input.to(device)

        mask = generate_encoder_input_mask(len(aids_input[0]), len(event_sequences), num_heads)
        mask = mask.to(device)
        Loss = model.train_step(aids_input, types_input, negative_samples, mask)
        optimizer.zero_grad()
        loss = Loss['loss']
        loss.backward()
        optimizer.step()

        total_loss += loss
        num_batch += 1
    del train_dataset
    gc.collect()
        # Save a checkpoint every `save_interval` batches
    checkpoint_path = f'checkpoint_{epoch}.pt'
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)

    # for root, dirs, files in os.walk('/kaggle/input/validation-data/'):
    #     for file in files:
    #         validation_files.append(os.path.join(root, file))
    validation_files=['validation_seq_5.npy','validation_seq_10.npy', 'validation_seq_15.npy', 'validation_seq_20.npy', 'validation_seq_25.npy',
                  'validation_seq_30.npy', 'validation_seq_35.npy', 'validation_seq_40.npy', 'validation_seq_50.npy', 'validation_seq_100.npy', 'validation_seq_150.npy']
    validation_dataset = NumpyDataset(validation_files, batch_size)
    batches = next(iter(validation_dataset))
    total_correct = 0
    total_dcg = 0
    total_seq = 0
    for batch in batches:
        event_sequences = batch['items']
        negative_samples = torch.from_numpy(batch['negatives'])

        aids_input = torch.from_numpy(np.array([event_sequence[0] for event_sequence in event_sequences]))
        types_input = torch.from_numpy(np.array([event_sequence[1] for event_sequence in event_sequences]))
        aids_input = aids_input.to(device)
        types_input = types_input.to(device)

        mask = generate_encoder_input_mask(len(aids_input[0]), len(event_sequences), num_heads)
        mask = mask.to(device)
        dcg, correct = model.validation_step(aids_input, types_input, mask)
        total_dcg += dcg
        total_correct += correct
        total_seq += len(event_sequences)

    ndcg = total_dcg / total_seq
    accuracy = total_correct / total_seq
    del validation_dataset
    gc.collect()
    print(f"ndcg: {ndcg}")
    print(f"accuracy: {accuracy}")
    time_duration = time.time() - start_time
    print(time_duration)
